# Sección 1: Carga y manipulación de datos en GPU con RAPIDS

## Carga de datos

In [ ]:
import os
import os.path

import cudf
import cupy as cp

Según el tamaño de muestra que deseemos estudiar, podemos cargar uno o múltiples datasets en memoria. Para evitar discrepancias en el número de columnas, seleccionamos manualmente las que nos interesan.

In [ ]:
#cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = ['amsterdam', 'antwerp', 'asheville', 'athens', 'austin', 'sevilla', 'shanghai'] #World 1
cities_to_use = ['amsterdam', 'antwerp', 'asheville', 'athens', 'austin', 'bangkok', 'sevilla', 'shanghai'] #World 2

columns_to_use = ['id', 'host_id', 'host_response_rate', 'host_acceptance_rate',
                  'latitude', 'longitude', 'accommodates', 'price', 'number_of_reviews', 'reviews_per_month',
                 'neighbourhood_cleansed']

Cargamos en memoria los datasets que vayamos a utilizar y los compilamos en uno. Convertimos columnas con tipos de datos dispares de modo que puedan ser almacenadas en un solo DataFrame.

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                if 'price' in temp_df.columns:
                    if(temp_df['host_acceptance_rate'].dtype != 'object'):
                        temp_df['host_acceptance_rate'] = temp_df['host_acceptance_rate'].astype('object')
                    if(temp_df['neighbourhood_cleansed'].dtype != 'object'):
                        temp_df['neighbourhood_cleansed'] = temp_df['neighbourhood_cleansed'].astype('object')
                    if listings.size == 0:
                        listings = temp_df
                    else:
                        for column in listings.columns:
                            if listings[column].dtype != temp_df[column].dtype:
                                print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                        listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index()
listings.shape

La instrucción nvidia-smi nos permite leer la información de uso de la(s) GPU(s) disponible(s). En particular, es útil para controlar cuánta memoria de vídeo (VRAM) está en uso. Dado que RAPIDS carga todos los datos en la GPU para optimizar el acceso a la información, hay que controlar que haya suficiente memoria antes de leer un fichero.

In [ ]:
!nvidia-smi

## Tratamiento de datos con cuDF y cuPY

Convertimos tipos numéricos en 64 bits a 32 u 8 bits, si es obvio que no alcanzarán valores extremos.

Dado que algunos algoritmos no suportan datos no numéricos, factorizamos las columnas de texto. La operación factorize() convierte una columna con valores no numéricos a un mapa en el que cada valor único se representa por un integer.

In [ ]:
%%time
listings['accommodates'] = listings['accommodates'].astype('int32')
listings['number_of_reviews'] = listings['number_of_reviews'].astype('int32')
listings['reviews_per_month'] = listings['reviews_per_month'].astype('float32').fillna(-1.0)

listings['neighbourhood_cleansed'], neighborhood_names = listings['neighbourhood_cleansed'].factorize()

En columnas con texto no deseado (como precios con símbolos de moneda), limpiamos caracteres no deseados y convertimos valores nulos.

In [ ]:
%%time
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '').fillna('-1').astype('int8')
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '').fillna('-1').astype('int8')
listings['price'] = listings['price'].str.replace(['$', ','], '').astype('float32')

## Generación de datos geográficos

Convertimos las coordenadas de longitud y latitud a distancias norte y este, a fin de normalizar la escala de los gráficos. La función de conversión (con su fuente de referencia) se encuentra en un fichero de utilidad aparte.

In [ ]:
%run ../utils/f_northing.py

In [ ]:
%%time
cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

Resultado final del tratamiento de datos:

In [ ]:
listings.dtypes

In [ ]:
listings.head()

# Sección 2: Carga y manipulación de datos en CPU con pandas y numPy

## Carga de datos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                if 'price' in temp_df.columns:
                    if(temp_df_cpu['host_acceptance_rate'].dtype != 'object'):
                        temp_df_cpu['host_acceptance_rate'] = temp_df_cpu['host_acceptance_rate'].astype('object')
                    if(temp_df_cpu['neighbourhood_cleansed'].dtype != 'object'):
                        temp_df_cpu['neighbourhood_cleansed'] = temp_df_cpu['neighbourhood_cleansed'].astype('object')
                    if listings_cpu.size == 0:
                        listings_cpu = temp_df_cpu
                    else:
                        for column in listings_cpu.columns:
                            if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                                print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                        listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index()
listings_cpu.shape

## Tratamiento de datos

In [ ]:
%%time
listings_cpu['accommodates'] = listings_cpu['accommodates'].astype('int32')
listings_cpu['number_of_reviews'] = listings_cpu['number_of_reviews'].astype('int32')
listings_cpu['reviews_per_month'] = listings_cpu['reviews_per_month'].astype('float32').fillna(-1.0)

listings_cpu['neighbourhood_cleansed'], neighborhood_names = listings_cpu['neighbourhood_cleansed'].factorize()

In [ ]:
%%time
listings_cpu['host_response_rate'] = listings_cpu['host_response_rate'].fillna('-1').str.replace('%', '').astype('int32')
listings_cpu['host_acceptance_rate'] = listings_cpu['host_acceptance_rate'].fillna('-1').str.replace('%', '').astype('int32')
listings_cpu['price'] = listings_cpu['price'].fillna('-1').str.replace('$', '').str.replace(',', '').astype('float32')

## Generación de datos geográficos

In [ ]:
%run ../utils/f_northing_numpy.py

In [ ]:
%%time
numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')

In [ ]:
listings.dtypes

In [ ]:
listings.head()

# Sección 3: Visualización de resultados mediante cuXfilter

La librería cuXfilter nos permite visualizar datos gráficamente. Vamos a visualizar los listados de AirBnB en Sevilla a 29 de octubre de 2020, con un selector por zonas.

Para ver el gráfico sin abrir un widget podemos usar el siguiente comando:

Para detener la ejecución del gráfico, podemos usar el siguiente comando: